# Cenário 📊

Acabo de ser contratado como Data Analyst pela empresa fictícia EcoStyle Co., um e-commerce de moda sustentável com operações no Brasil e na Europa.

<br>

## Contexto do negócio

Modelo => Venda online de roupas e acessórios ecológicos (B2C) <br>
Objetivo em 2025 => Aumentar a receita em 25 %, reduzir churn para < 5 % ao mês, otimizar investimento em marketing digital
<br>
Desafio atual: Crescemos rápido; agora precisamos saber onde investir e quais produtos/mercados impulsionam (ou travam) o crescimento.

### 🗂️ Dados fornecidos

*** Estão em formato CSV; você pode carregar com pd.read_csv(). ***
- sales_2024.csv;
- customers.csv;
- marketing_spend.csv;
- site_traffic.csv


#### 🔍Perguntas que devem ser respondidas

.📈 Como evoluíram as receitas mensais em 2024?
.🏆 Quais 10 sub-categorias geraram mais receita e margem? 
.💸 Qual canal traz melhor retorno (Revenue / Spend)?
.🔄 Qual a taxa de recompra por coorte de mês de aquisição?
.🕒 Há horário/dia com maior taxa de conversão?

-------------------------

Cronograma ⏱️

Exploração & limpeza	até 03 Jun 2025
Visualizações preliminares	06 Jun 2025
Relatório final + recomendações	10 Jun 2025
